The objective here is to attempt to cluster all the conversation snippets we have from ec-council into clusters, then observe these clusters to see what kind of pattern the clustering happened on. We are going to use K-means as our clustering algorithm and we are going to use universal sentence encoder as the embedding model.
The steps are as follows:
* Fetch all the snippets for a manager, say geetha
* Encode all of them
* Pass them to a clusterer
* Observe

In [9]:
class VadChunk(object):
    """Represents a snippet of the audio file post performing vad."""

    def __init__(self, sid, path, from_time, to_time, speaker, text, confidence, words, encoding=None):
        self.sid = sid
        self.path = path
        self.from_time = from_time
        self.to_time = to_time
        self.speaker = speaker
        self.text = text
        self.confidence = confidence
        self.words = words
        self.encoding = encoding

    def set_sid(self, sid):
        self.sid = sid

    def set_use_encoding(self, encoding):
        self.encoding = encoding

In [2]:
import os
from os import sys
from os import path

In [5]:
sys.path.append(path.dirname(path.dirname(os.getcwd())))
from speech.utils.db import execute_query

In [6]:
sql = "select * from snippet where task_id in (select task.id from task where task.owner = 218927)"
rows, colnames = execute_query(sql)

/home/absin/git/ai/speech/utils/constants.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  doc = yaml.load(f)


In [10]:
vad_chunks = []
for row in rows:
    vad_chunks.append(VadChunk(row[colnames.index('id')], None, 
                               row[colnames.index('from_time')], row[colnames.index('to_time')], 
                               row[colnames.index('speaker')], row[colnames.index('text_')], row[colnames.index('confidence')], 
                               []))

In [13]:
vad_chunks[0].__dict__

{'sid': 529,
 'path': None,
 'from_time': 9.15,
 'to_time': 15.54,
 'speaker': 'Agent',
 'text': 'aap kya',
 'confidence': 0.69784289598465,
 'words': [],
 'encoding': None}

In [33]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

/home/absin/git/ai/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/absin/git/ai/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/absin/git/ai/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/absin/git/ai/venv/lib/python3.6/site-packages/tensorflow/python/framework

In [34]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [35]:
messages = []
message_embeddings = []
for vad_chunk in vad_chunks:
    messages.append(vad_chunk.text)

In [36]:
embed = hub.Module(module_url)
tf.logging.set_verbosity(tf.logging.ERROR)

ResourceExhaustedError: OOM when allocating tensor with shape[57142,128,512] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node module_apply_default/Encoder_en/Transformer/PrepareForTransformer/GatherV2 (defined at /home/absin/git/ai/venv/lib/python3.6/site-packages/tensorflow_hub/native_module.py:561) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Original stack trace for 'module_apply_default/Encoder_en/Transformer/PrepareForTransformer/GatherV2':
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/absin/git/ai/venv/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/absin/git/ai/venv/lib/python3.6/site-packages/traitlets/config/

In [55]:
message_embeddings

['h a s n i',
 'ok',
 'ok',
 'h',
 'ok',
 'CH',
 'ok',
 'yahoo.com',
 ' you have given one email ID of Gmail that is not working.',
 'Maya',
 'aap kya',
 'Hello.',
 'Are you interested for that program?',
 'yeah, you want to',
 'Ok, please tell me so what what information you need from my side?',
 'What do you offer?',
 'We are offering you so we are coming.',
 'Can you set?',
 'would his',
 ' tell me your email ID']

In [57]:
index = 0
unit_work = 1000
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  while index<len(messages):
        unit_work_messages = messages[index:index+unit_work]
        index+=unit_work
        message_embeddings_unit = session.run(embed(unit_work_messages))
        message_embeddings.extend(message_embeddings_unit)
        print('finished {}'.format(index))

finished 1000
finished 2000
finished 3000
finished 4000
finished 5000
finished 6000
finished 7000
finished 8000
finished 9000
finished 10000
finished 11000
finished 12000
finished 13000
finished 14000
finished 15000
finished 16000
finished 17000
finished 18000
finished 19000
finished 20000
finished 21000
finished 22000
finished 23000
finished 24000
finished 25000
finished 26000
finished 27000
finished 28000
finished 29000
finished 30000
finished 31000
finished 32000
finished 33000
finished 34000
finished 35000
finished 36000
finished 37000
finished 38000
finished 39000
finished 40000
finished 41000
finished 42000
finished 43000
finished 44000
finished 45000
finished 46000
finished 47000
finished 48000
finished 49000
finished 50000
finished 51000
finished 52000
finished 53000
finished 54000
finished 55000
finished 56000
finished 57000
finished 58000


In [71]:
message_embeddings_np = np.array(message_embeddings)

In [73]:
X.shape

(300, 2)

In [72]:
message_embeddings_np.shape

(57142, 512)

In [74]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20)
kmeans.fit(message_embeddings_np)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=20, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [27]:
kmeans.__dict__

{'n_clusters': 4,
 'init': 'k-means++',
 'max_iter': 300,
 'tol': 0.0001,
 'precompute_distances': 'auto',
 'n_init': 10,
 'verbose': 0,
 'random_state': None,
 'copy_x': True,
 'n_jobs': None,
 'algorithm': 'auto',
 'cluster_centers_': array([[ 0.94973532,  4.41906906],
        [-1.37324398,  7.75368871],
        [-1.58438467,  2.83081263],
        [ 1.98258281,  0.86771314]]),
 'labels_': array([3, 1, 0, 1, 3, 3, 2, 0, 1, 1, 2, 1, 0, 1, 3, 0, 0, 3, 2, 2, 3, 3,
        0, 2, 2, 0, 3, 0, 2, 0, 1, 1, 0, 1, 1, 1, 1, 1, 2, 3, 0, 2, 0, 0,
        2, 2, 1, 2, 1, 3, 2, 3, 1, 3, 3, 2, 1, 2, 1, 3, 1, 0, 1, 2, 2, 2,
        1, 3, 1, 2, 0, 2, 1, 2, 2, 1, 2, 0, 3, 1, 3, 0, 3, 3, 1, 0, 3, 0,
        1, 1, 0, 3, 1, 2, 2, 0, 3, 3, 0, 2, 1, 3, 1, 3, 0, 3, 3, 0, 1, 0,
        2, 2, 3, 1, 3, 0, 1, 3, 3, 0, 2, 3, 2, 3, 3, 3, 3, 2, 3, 2, 1, 2,
        2, 3, 1, 2, 2, 1, 0, 1, 1, 2, 0, 2, 0, 2, 1, 0, 1, 1, 1, 0, 1, 0,
        3, 2, 1, 2, 3, 0, 1, 0, 0, 3, 0, 2, 2, 0, 3, 0, 0, 1, 3, 0, 2, 1,
        3, 3, 0

In [77]:
kmeans.labels_.shape

(57142,)

In [102]:
for i, vad_chunk in enumerate(vad_chunks):
    if kmeans.labels_[i] == 19:
        print('{} ---> [{}]:{}:{}'.format(kmeans.labels_[i],vad_chunks[i].speaker, vad_chunks[i].sid, vad_chunks[i].text))

19 ---> [Agent]:744:Yes, you just call you to understand because we are coming to conduct 4 days master classroom program on Certified Ethical hacking in Jeddah in one of the star.
19 ---> [Customer]:754:password in this training
19 ---> [Agent]:968:got connected through internet related to our upcoming cyber security program Certified Ethical hacking in USA
19 ---> [Agent]:979:Yeah, yeah, this is a training on certified ethical hacker.
19 ---> [Agent]:1002:Is in regards upcoming cyber security?
19 ---> [Agent]:627:This is your interested that you sentence on LinkedIn ad regarding upcoming cyber security training programs in Muscat Oman right?
19 ---> [Customer]:629:cyber security
19 ---> [Agent]:633:You know in Oman in Oman understand you work with information technology authority write it authority women.
19 ---> [Agent]:635:Ok, ok you just started raining there for 2 months ok, no problem I get it's ok you. So also, are you interested in taking up this training program because this 

19 ---> [Agent]:1191159:information to the startup problem that I will
19 ---> [Agent]:1191193:Access to R10 blocked in lab where in you can I know where you can we can develop blockchain program.
19 ---> [Agent]:1191446:If you look at it smart and expensive investment image, please consider this as an investment for a guy who doesn''t know who has no no not much animals or no voice. No, very limited information of blockchains want to complete this complete Innova sensation on also ine go to the all the all the information that we are providing online also access to the left you will be able to develop and deploy project on your own.
19 ---> [Agent]:1191632:water soluble learning in the program what are the models that to be covered and also in what ways is this model should be covered there are three pages for this particular program where in first page will be talking about the introduction of blockchain in the second page will be talking about the
19 ---> [Agent]:1191673:learning le

Let's recluster using the same algorithm and the same number of clusters but this time we are going to skip the snippets which have low confidence

In [106]:
vad_chunks_high_confidence = []
message_embeddings_high_confidence = []
for index, vad_chunk in enumerate(vad_chunks):
    if vad_chunk.confidence > 0.9:
        vad_chunks_high_confidence.append(vad_chunk)
        message_embeddings_high_confidence.append(message_embeddings[index])
message_embeddings_high_confidence_np = np.array(message_embeddings_high_confidence)

In [111]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20)
kmeans.fit(message_embeddings_high_confidence_np)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=20, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [131]:
for i, vad_chunk in enumerate(vad_chunks_high_confidence):
    if kmeans.labels_[i] == 20:
        print('{} ---> [{}]:{}:{}'.format(kmeans.labels_[i],vad_chunk.speaker, vad_chunk.sid, vad_chunk.text))

LSH

In [132]:
!pip install FALCONN

     |████████████████████████████████| 1.4MB 1.4MB/s eta 0:00:01
  Created wheel for FALCONN: filename=FALCONN-1.3.1-cp36-cp36m-linux_x86_64.whl size=10582310 sha256=e108de0c4e581322f27f31ce952c181b30da4cd93238439bb177b68706b5ec86
  Stored in directory: /home/absin/.cache/pip/wheels/bf/36/96/d5538901888620fc0343c1ed9d5f87fce00869e00c12056ef8
Successfully built FALCONN
You should consider upgrading via the 'pip install --upgrade pip' command.


In [133]:
import falconn